In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates
import scipy
import numpy as np
from pylab import *
from scipy.optimize import curve_fit
from scipy.stats import linregress

In [93]:
df=pd.read_excel('SN.xlsx')

In [94]:
df.tail()

,SN,m15B,Em15B,B,EB,V,Ev,i,Ei
1,1986G,1.73,0.07,-17.72,0.42,-18.12,0.32,-18.21,0.23
2,1989B,1.31,0.07,-18.50,0.33,-18.50,0.32,-18.30,0.31
3,1991T,0.94,0.07,-18.96,0.31,-19.10,0.31,-19.04,0.30
4,1991bg,1.88,0.10,-16.38,0.15,-17.13,0.10,-17.57,0.10
5,1992A,1.33,0.04,-18.05,0.14,-18.10,0.13,-17.85,0.12


In [95]:
#Método de York
#Y=B+Ax+e
B=-19.891
A=1.187

In [96]:
ox=df.Em15B
oy=df.Ei

In [97]:
Wi=pd.DataFrame([])

In [98]:
#correlación entre los errores de x, y
r=[]
for i in range(0,len(df)):
    g=(df.Em15B[i]-mean(df.Em15B))*(df.Ei[i]-mean(df.Ei))
    h=sqrt((df.Em15B[i]-mean(df.Em15B))**2)
    k=sqrt((df.Ei[i]-mean(df.Ei))**2)
    r.append(g/(h*k))

In [99]:
w=[]
a= sqrt(ox*oy)
for i in range(0,len(df)):
   # if i==14:
    #    k=0
    #else:
    j=(ox[i] * A**2 * oy[i] - 2*A * a[i] * r[i] )
    c = ox[i]*oy[i]#/(ox[i] * A**2 * oy[i] - 2*A * a[i] * r )
    k= c/j
    w.append(k)

In [100]:
#Parámetros de la linea ajustada al error
#y=Bo+Aox
Bo=0
Ao=0

U=np.std(df.m15B)
V=np.std(df.i)

In [101]:
s=[]
j=0
k=0

for i in range(0, len(df)):
    #if i==14:
       # b=0
       # j=( w[i] * b * V ) + j
       # k=( w[i] * b * U) + k
    #else:
    b=w[i]*( (U/oy[i]) + (B*ox[i]/V) - (B*U+V)*(r[i]/a[i]) )
    j=( w[i] * b * V ) + j
    k=( w[i] * b * U) + k

Ao=j/k
Ao

1.4701568209690392

In [102]:
Bo=mean(df.i)-Ao*mean(df.m15B)
Bo

-20.290371378934626

In [103]:
#Calcular el error
oA=0
j=0
for i in range(0, len(df)):
    j=w[i]*U**2
    
    
oA=sqrt(-1/j)
oA

18.476441370717954

In [104]:
oB=0
j=0
for i in range(0, len(df)):
    j=w[i]
oB=sqrt(j+mean(df.m15B)*oA**2)
oB

21.951828136948894

In [105]:
#calculo de S
S=0
#z=mean(a)
for i in range(0, len(df)):
    S= w[i]*( df.i[i]- df.m15B[i]*Ao-Bo)**2 + S  
S=abs(S)

In [106]:
eBo=oB*sqrt(S/(len(df)-2))

eAo=oA*sqrt(S/(len(df)-2))

In [107]:
eAo

1.3452975488167491

In [108]:
eBo

1.5983456982948416

In [109]:
#R de Pearson
RSS=0
for i in range(0, len(df)):
    RSS=(A+B*df.m15B[i]-df.i[i])**2 / ((.147*Ao)**2 + (.1788)**2) + RSS
#RSS

In [110]:
TSS=0
for i in range(0, len(df)):
    TSS=(df.i[i]-mean(df.i))**2+TSS
#TSS

In [111]:
1-(TSS/RSS)

0.9998614576435236